In [4]:
import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

In [31]:
train_input, train_target, test_input, test_target = \
prologue.load_data(one_hot_labels = True, normalize = True, flatten = False)
                                                                        
train_target.size()                                                                        
                                                                        

                                                                        
                                                                        

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


torch.Size([1000, 10])

In [44]:
class CNet(nn.Module):
    def __init__(self):
        super(CNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=4)
        self.conv2 = nn.Conv2d(20, 800, kernel_size=5)
        self.fc1 = nn.Linear(360,150)
        self.fc2 = nn.Linear(150,10)
        
    def forward(self,x):
        x = F.torch.tanh(F.max_pool2d(self.conv1(x), kernel_size=2, stride=1))
        x = F.torch.tanh(F.max_pool2d(self.conv2(x), kernel_size=3, stride=1))
        x = F.torch.tanh(self.fc1(x.view(-1, 360)))
        x = F.softmax(self.fc2(x))
        return x    

In [45]:
def train_model(model, train_input, train_target, mini_batch_size):
    criterion = nn.MSELoss()
    eta = 1e-1
    epochs = 50
    
    for e in range(epochs):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            output.size()
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            sum_loss = sum_loss + loss.item()
            for p in model.paramenters():
                p.data.sub_(eta*p.grad.data)
        print(e, sum_loss)
        
        
def compute_nb_errors(model, input, target, mini_batch_size):
    for b in range (0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        predicted_classes = output.data.max(1)
        for k in range(mini_batch_size):
            if target.data[b + k, predicted_classes[k]] <= 0:
                nb_errors = nb_errors+1
    return nb_errors
                    
    
    

In [46]:
train_input, train_target = Variable(train_input), Variable(train_target)
test_input, test_target = Variable(test_input), Variable(test_target)

mini_batch_size = 50

In [47]:
for k in range(10):
    model = CNet()
    train_model(model, train_input, train_target, mini_batch_size)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))

RuntimeError: The size of tensor a (36000) must match the size of tensor b (50) at non-singleton dimension 0